In [116]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import link
from IPython.display import display, clear_output, Math
from math import sqrt, pi
from scipy.integrate import solve_ivp

In [117]:
# --- core formulas ---
def omega0(L, C): return 1.0/np.sqrt(L*C)
def zeta(R, L, C): return (R/2.0)*np.sqrt(C/L)
def compute_poles(R, L, C):
    w0, z = omega0(L, C), zeta(R, L, C)
    if z < 1.0:
        wd = w0*np.sqrt(1.0 - z*z)
        return complex(-z*w0, +wd), complex(-z*w0, -wd)
    # z >= 1: real poles
    root = np.sqrt(max(0.0, z*z - 1.0))
    s1 = -w0*(z - root)
    s2 = -w0*(z + root)
    return complex(s1, 0.0), complex(s2, 0.0)

def H_jw(R, L, C, w):
    Zc = 1.0/(1j*w*C)
    return Zc / (R + 1j*w*L + Zc)   # Vc/Vin

In [118]:
# --- simulations ---
# Simulate RLC step response using ODE solver
def simulate_step_Vc_only(R, L, C, V_step, t_end, n=3000):
    if t_end <= 0: return np.array([]), np.array([])
    def f(t, x):
        q, i = x
        return [i, (V_step - R*i - q/C)/L]
    t = np.linspace(0, t_end, n)
    sol = solve_ivp(f, [0, t_end], [0.0, 0.0], t_eval=t, max_step=t_end/max(n,1))
    if not sol.success: return t, np.full_like(t, np.nan)
    return t, sol.y[0] / C  # Vc = q/C

# Analytic solution for RLC step response
def analytic_Vc(Vin, R, L, C, t):
    w0, z = omega0(L, C), zeta(R, L, C)
    if z < 1.0:
        wd  = w0*np.sqrt(1.0 - z*z)
        A   = 1.0/np.sqrt(1.0 - z*z)
        phi = np.arccos(z)
        return Vin*(1 - A*np.exp(-z*w0*t)*np.sin(wd*t + phi))
    elif abs(z-1.0) <= 1e-12:
        return Vin*(1 - (1 + w0*t)*np.exp(-w0*t))
    else:
        s1 = -w0*(z - np.sqrt(z*z - 1.0))
        s2 = -w0*(z + np.sqrt(z*z - 1.0))
        return Vin*(1 - ((s2*np.exp(s1*t) - s1*np.exp(s2*t))/(s2 - s1)))


In [119]:
# --- plotting ranges  ---
# choose time samples based on t_end
def choose_time_samples(t_end):
    return 1500 if t_end < 5e-3 else 3000 if t_end < 5e-2 else 5000 if t_end < 0.2 else 8000

# choose frequency sweep based on w0 and z
def choose_freq_sweep(w0, z):
    span = 100.0 if (z < 0.5) else 40.0 if (z < 1.0) else 20.0
    wmin = max(1e-3, w0/span); wmax = w0*span
    Nf   = 1200 if span >= 40 else 800
    return np.logspace(np.log10(wmin), np.log10(wmax), Nf)

# --- formatting float number ---
def _fmt(x, precision=6):
    s = np.format_float_positional(float(x), precision=precision, trim='-')
    if s.startswith('-0') and round(float(x), precision) == 0.0:
        s = '0'
    return s

In [120]:
# Display numeric formulas
def show_numeric_formulas(R, L, C, Vin, s1, s2):
    w0 = omega0(L, C)
    z  = zeta(R, L, C)
    def tex(s):
        display(Math(s))

    # UNDERDAMPED: ζ < 1 \
    if z < 1.0:
        wd  = abs(s1.imag)                         # damped natural freq
        A   = 1.0/np.sqrt(1.0 - z*z)               # amplitude scale in step response
        phi = np.arccos(z)                         # phase shift
        Q   = 1.0/(2.0*z)                          # quality factor
        Mp  = np.exp(-np.pi*z/np.sqrt(1.0 - z*z))  # overshoot fraction
        Ts  = 4.0/(z*w0)                           # ~ settling time
        
        # 상태 + 핵심 파라미터
        tex(
            r"\textbf{UNDERDAMPED }(\zeta<1)"
            + r"\quad \zeta=" + _fmt(z)
            + r",\ \omega_0=" + _fmt(w0) + r"\ \text{rad/s}"
            + r",\ Q=" + _fmt(Q)
        )
        # 시간 응답 Vc(t)
        tex(
            r"V_C(t)=" + _fmt(Vin)
            + r"\Big[1-"
            + _fmt(A)
            + r" e^{-("+ _fmt(z*w0) + r")t}"
            + r"\sin(" + _fmt(wd) + r"t+" + _fmt(phi) + r")\Big]"
        )
        # 핵심 성능 지표
        tex(
            r"M_p \approx "
            + f"{Mp*100:.2f}"
            + r"\% \quad,\quad "
            + r"T_s \approx "
            + _fmt(Ts)
            + r"\ \text{s}"
        )
    # CRITICALLY DAMPED: ζ = 1 → 중복 실근, 진동 없이 가장 빠르게 감쇠
    elif abs(z - 1.0) < 0.02: # zeta가 1에 매우 근접한 경우
        tau = 1.0/w0  # dominant time constant
        tex(
            r"\textbf{CRITICALLY DAMPED }(\zeta=1)"
            + r"\quad \zeta=" + _fmt(z)
            + r",\ \omega_0=" + _fmt(w0) + r"\ \text{rad/s}"
            + r",\ \tau=" + _fmt(tau) + r"\ \text{s}"
        )
        tex(
            r"V_C(t)=" + _fmt(Vin)
            + r"\Big[1-(1+" + _fmt(w0)
            + r"t)e^{-(" + _fmt(w0) + r")t}\Big]"
        )
        tex(r"\text{no overshoot, fastest return}")

    # OVERDAMPED: ζ > 1 → 서로 다른 두 실근, 진동 없이 느리게 감쇠
    else:
        s1r, s2r = s1.real, s2.real
        tau_slow = 1.0/abs(min(s1r, s2r, key=lambda x: abs(x)))
        tex(
            r"\textbf{OVERDAMPED }(\zeta>1)"
            + r"\quad \zeta=" + _fmt(z)
            + r",\ \omega_0=" + _fmt(w0) + r"\ \text{rad/s}"
            + r",\ \tau_{\text{slow}}\approx " + _fmt(tau_slow) + r"\ \text{s}"
        )
        tex(
            r"V_C(t)=" + _fmt(Vin)
            + r"\left[1-\frac{(" + _fmt(s2r) + r")e^{("
            + _fmt(s1r) + r")t}-(" + _fmt(s1r)
            + r")e^{(" + _fmt(s2r)
            + r")t}}{" + _fmt(s2r - s1r) + r"}\right]"
        )
        tex(r"\text{no oscillation, slow tail}")

In [121]:
# --- 입력 UI ---
# R (Ohm)
R_slider = widgets.FloatSlider(
    description='R (Ω)',
    min=0.1, max=100.0, step=0.1,
    value=10.0,
    readout=True,
    continuous_update=True,  # 드래그 중에도 업데이트
    layout=widgets.Layout(width='250px')
)
# L (H) - 슬라이더
L_slider = widgets.FloatLogSlider(
    description='L (H)',
    base=10,
    min=-6,  # 1e-6 H
    max=-1,  # 1e-1 H
    step=0.1,
    value=1e-3,
    readout=True,
    layout=widgets.Layout(width='250px')
)
# C (F) - 슬라이더
C_slider = widgets.FloatLogSlider(
    description='C (F)',
    base=10,
    min=-9,  # 1e-9 F
    max=-4,  # 1e-4 F
    step=0.1,
    value=1e-5,
    readout=True,
    layout=widgets.Layout(width='250px')
)

VIN_in  = widgets.FloatText(value=1.0, description='Vin_step [V]', layout=widgets.Layout(width='170px'))
TEND_in = widgets.FloatText(value=5e-3, description='t_end [s]', layout=widgets.Layout(width='150px'))
out     = widgets.Output()



In [122]:
def main(_=None):
    with out:
        clear_output()
        R = R_slider.value
        L = L_slider.value
        C = C_slider.value
        Vin, t_end = VIN_in.value, TEND_in.value
        if L<=0 or C<=0 or R<0 or t_end<=0:
            print("Invalid parameters: require L>0, C>0, R>=0, t_end>0."); return

        # (1) numeric formulas (with substituted values)
        s1, s2 = compute_poles(R, L, C)
        show_numeric_formulas(R, L, C, Vin, s1, s2)

        # (2) time response: numeric vs analytic
        n_time = choose_time_samples(t_end)
        t, Vc_num = simulate_step_Vc_only(R, L, C, Vin, t_end, n_time)
        Vc_ana = analytic_Vc(Vin, R, L, C, t)
        fig1, ax1 = plt.subplots(figsize=(6.6,3.2))
        ax1.plot(t, Vc_num, label='Numeric (solve_ivp)')
        ax1.plot(t, Vc_ana, '--', label='Analytic')
        ax1.set_title("Step response: Vc(t)")
        ax1.set_xlabel("t [s]"); ax1.set_ylabel("Vc [V]")
        ax1.grid(True, ls=':'); ax1.legend(); plt.tight_layout(); 
        plt.show()
for w in [R_slider, L_slider, C_slider, VIN_in, TEND_in]:
    w.observe(main, names='value')
main()  
    
display(widgets.VBox([
    widgets.HTML("<h4>DC Step — R,L,C,Vin_step,t_end (compact)</h4>"),
    widgets.HBox([R_slider, L_slider,C_slider, VIN_in, TEND_in]),
    out
]))
        

In [123]:
# --- (1) AC 시뮬레이터: 정현파 입력 Vin = Vac * sin(ωt) ---
def show_ac_summary(R, L, C, Vac, w_drive):

    # 시스템 파라미터
    w0 = omega0(L, C)
    z  = zeta(R, L, C)
    # 전달함수 H(jω) = Vc/Vin
    H_drive = H_jw(R, L, C, w_drive)
    mag     = np.abs(H_drive)           # |H|
    phi     = np.angle(H_drive)         # [rad]

    def tex(s):
        display(Math(s))
    # 요약 텍스트
    tex(
        r"V_C^{\text{(steady)}}(t) \approx "
        + _fmt(mag * Vac)
        + r"\,\sin\!\big("
        + _fmt(w_drive)
        + r"t + "
        + _fmt(phi)
        + r"\big)"
    )
    tex(
        r"|H| = " + _fmt(mag)
        + r",\quad"
        r"\angle H = " + _fmt(phi)
        + r"\ \mathrm{rad}"
        + r"\ (" + _fmt(phi * 180/np.pi) + r"^\circ)"
    )

def simulate_ac_Vc(R, L, C, Vac, w_drive, t_end, n=3000):
    """
      dq/dt = i
      di/dt = (Vin(t) - R i - q/C)/L
      Vin(t) = Vac * sin(w_drive * t)
    """
    def f(t, x):
        q, i = x
        Vin_t = Vac * np.sin(w_drive * t)
        dqdt = i
        didt = (Vin_t - R*i - q/C)/L
        return [dqdt, didt]
    # 시간축
    t = np.linspace(0, t_end, n)

    q0 = 0.0    # => Vc(0)=0
    i0 = 0.0
    sol = solve_ivp(
        f,
        [0, t_end],
        [q0, i0],
        t_eval=t,
        max_step=t_end/max(n, 1)
    )
    if not sol.success:
        return t, np.full_like(t, np.nan), np.full_like(t, np.nan)
    q = sol.y[0]
    Vc = q / C
    Vin_t = Vac * np.sin(w_drive * t)
    return t, Vc, Vin_t


# --- (2) AC 입력 관련 위젯 ---
VAC_in   = widgets.FloatText(
    value=1.0,
    description='Vac [V]',
    layout=widgets.Layout(width='150px')
)
FREQ_slider = widgets.FloatLogSlider(
    description='f (Hz)',
    base=10,
    min=1, max=6, step=0.05,
    value=1e3
)
TAC_in   = widgets.FloatText(
    value=5e-3,
    description='t_end_AC [s]',
    layout=widgets.Layout(width='170px')
)
out_ac = widgets.Output()

# --- (3) AC 전용 main 함수 ---
def main_ac(_=None):
    with out_ac:
        clear_output()
        # --- RLC / AC 파라미터 읽기 ---
        R   = R_slider.value
        L   = L_slider.value
        C   = C_slider.value
        Vac = VAC_in.value           # [V]   사인파 진폭
        f_drive = FREQ_slider.value  # [Hz]  구동 주파수
        w_drive = 2.0 * np.pi * f_drive  # [rad/s]
        t_end_ac = TAC_in.value     # [s]   시뮬레이션 길이
        # --- 유효성 검사 ---
        if L <= 0 or C <= 0 or R < 0:
            print("Invalid RLC: need L>0, C>0, R>=0.")
            return
        if t_end_ac <= 0 or f_drive < 0:
            print("Invalid AC params: need t_end>0 and f>=0.")
            return
        # --- 적당한 샘플 수 정하기
        n_time_ac = choose_time_samples(t_end_ac)
        # ====================================================
        # --- 정상상태(해석적) 표현을 먼저 보여주기
        show_ac_summary(R, L, C, Vac, w_drive)
        # --- 실제(수치) 시뮬레이션: 과도 + 정상상태 포함
        t_ac, Vc_ac, Vin_ac = simulate_ac_Vc(
            R, L, C,
            Vac,
            w_drive,
            t_end_ac,
            n_time_ac
        )
        # ====================================================
        # (1) 전체 시간 파형: Vin vs Vc
        fig_full, ax_full = plt.subplots(figsize=(6.6, 3.2))
        ax_full.plot(t_ac, Vin_ac, label='Vin(t) = Vac·sin(ωt)')
        ax_full.plot(t_ac, Vc_ac, '--', label='Vc(t)')
        ax_full.set_title("AC driven response (pure sine) - full window")
        ax_full.set_xlabel("t [s]")
        ax_full.set_ylabel("Voltage [V]")
        ax_full.grid(True, ls=':')
        ax_full.legend()
        plt.tight_layout()
        plt.show()

        # (2) 주파수 응답 크기 (Bode magnitude)
        w0 = omega0(L, C)
        z  = zeta(R, L, C)
        ws = choose_freq_sweep(w0, z)     # log-spaced ω range
        Hs = H_jw(R, L, C, ws)
        mag_db = 20*np.log10(np.abs(Hs) + 1e-30)
        fig_mag, ax_mag = plt.subplots(figsize=(6.4, 3.2))
        ax_mag.semilogx(ws, mag_db)
        ax_mag.axvline(w0, alpha=0.35, ls='--', label='ω0')
        ax_mag.set_title("Frequency response: |Vc/Vin| [dB]")
        ax_mag.set_xlabel("ω [rad/s]")
        ax_mag.set_ylabel("20 log10 |H(jω)| [dB]")
        ax_mag.grid(True, which='both', ls=':')
        ax_mag.legend()
        plt.tight_layout()
        plt.show()

        # (3) 주파수 응답 위상 (Phase)
        phi = np.unwrap(np.angle(Hs))          # [rad]
        fig_phase, ax_phase = plt.subplots(figsize=(6.4, 3.2))
        ax_phase.semilogx(ws, np.degrees(phi))
        ax_phase.axvline(w0, alpha=0.35, ls='--', label='ω0 (~ -90°)')
        ax_phase.set_title("Phase response: ∠H(jω)")
        ax_phase.set_xlabel("ω [rad/s]")
        ax_phase.set_ylabel("phase [deg]")
        ax_phase.grid(True, which='both', ls=':')
        ax_phase.legend()
        plt.tight_layout()
        plt.show()

# --- (4) 디스플레이: 위젯 배치 ---
for w in [R_slider, L_slider, C_slider, VAC_in, FREQ_slider, TAC_in]:
    w.unobserve_all()
    w.observe(main_ac, names='value')

main_ac()
display(
    widgets.VBox([
        widgets.HTML("<h4>AC drive settings</h4>"),
        widgets.HBox([VAC_in, FREQ_slider, TAC_in]),
        out_ac
    ])
)